In [8]:
import numpy as np

## 2.3. Circuit of the VCO in the HEF4046BT

For solving the exercise we used the following
[HEF4046BT datasheet](https://assets.nexperia.com/documents/data-sheet/HEF4046B.pdf) (last accessed on 10. May)

### 2.3.1 What is the purpose of $R_1$, $R_2$ and $C_1$ ?

According to the datasheet $R_1$, $R_2$ and $C_1$ are connected to the pins 11, 12 and for $C_1$ its across 6 and 7. <br>
$R_1$ and $C_1$ determine the frequency range of the VCO and $R_2$ can be used to set a frequency offset. <br>
The recommended range for the $R_1$ and $R_2$: $10k\Omega \leq R \leq 1M\Omega$. <br>
For the capacitor: $50pF \leq C$

### 2.3.2 Determine $R_1$ and $C_1$ for max. operating range between $50kHz$ and $200kHz$

Given values: <br>
- $R_2 = 100k\Omega$
- $V_{DD} = 5V$ <br>

Taking Fig.7 from the datasheet to determine $C_1$. <br>
Using Line 6 as it calculated with the given values, we get for $f_{min}=50kHz$ a value of $C_1 = 15 \cdot 10^3 \cdot 10^{-12}F = 15nF$. The screenshot is below. <br>
For determining the $R_1$ value, we take Fig. 8 and calculate the ratio $\frac{f_{max}}{f_min} = \frac{200kHz}{50kHz} = 4 $. <br>
From the grap we get for the ratio $\frac{R_2}{R_1} = 4$. So $R_1 = 24k\Omega$. The screenshot is also below. <br>
![title](f_min_c2.png) <br>
![title](r2_r1_bw.png) <br>

## 2.4. Frequency divider of the ADF 4113

For solving the exercise we used the following
[ADF4113 datasheet](https://www.analog.com/media/en/technical-documentation/data-sheets/ADF4110_4111_4112_4113.pdf) (last accessed on 10. May)

### 2.4.1 How must the divider ratio $n$ be chosen if $1.7GHz$, $1.75GHz$ and $1.8 GHz$ are to be output?

In [9]:
f_ref = 10e6
f = np.array([1.7e9, 1.75e9, 1.8e9])

n = np.divide(f, f_ref)
print(f"The divider ratios are: {n}")

The divider ratios are: [170. 175. 180.]


### 2.4.2 Can $1.755GHz$ also be realized? What divider ratios must / would then be used and how long must / would they be applied in relation to each other?

Yes, with fractional-N dividers, it is possible to achieve higher frequency resolution than with only integer-N PLLs. 
These dividers allow the division ratio to be a fractional number, not just an integer. 
Instead of using a fixed divider N, the system rapidly alternates between two integer divider values, N and N+1, for short periods of time.
The effective division factor (n_eff) is then the time-weighted average of the two dividers:

$$
n_{\text{eff}} = n + \frac{T_{n+1}}{T_n + T_{n+1}}
$$    

To realize the 1.755 GHz the devicer ratios can be calculated as the follwing.

In [ ]:
def calc_neff(n: int, Tn: float, Tn1: float) -> float:
    """
    Calculate the effective divider ratio for a fractional-N PLL.

    Args:
        n (int): Base divider value (integer part).
        Tn (float): Time duration using divider n.
        Tn1 (float): Time duration using divider n+1.

    Returns:
        float: Effective divider value n_eff.
    """
    n_eff = n + Tn1 / (Tn + Tn1)
    return n_eff

def calc_feff(n: int, Tn: float, Tn1: float) -> float:
    """
    Calculate the effective frequency at the output of the VCO
    
    Args: 
        n (int): Base divider value (integer part).
        Tn (float): Time duration using divider n.
        Tn1 (float): Time duration using divider n+1.
    """
    f_eff = (n * Tn + (n+1) * Tn1) * 10e6 
    return f_eff

# Target: n_eff = 175.5
n = 175
Tn1 = 0.5  # Time at n+1
Tn = 0.5   # Time at n

n_eff = calc_neff(n, Tn, Tn1)
print(f"Effective divider n_eff = {n_eff}")
f_eff = calc_feff(n, Tn, Tn1) / 1e6
print(f"Effective output frequency = 175.5 x 10 MHz = {f_eff:.1f} Mhz")

Effective divider n_eff = 175.5
Effective output frequency = 175.5 x 10 MHz = 1755.0 Mhz


The resulting devider rations are 175 and 176 with duty cycles of 50%.

### 2.4.3 What are / would be the values at 1.752 GHz?

The effective necessary division radtio is 1752 MHz / 10 MHz = 157.2
To calculate the time duratio for each division rate, we use the functions from above.

In [16]:
n = 175
Tn1 = 0.2
Tn = 0.8
n_eff = calc_neff(n, Tn, Tn1)
print(f"Effective divider n_eff = {n_eff}")
f_eff = calc_feff(n, Tn, Tn1) / 1000000
print(f"Effective output frequency = 175.5 x 10 MHz = {f_eff:.1f} Mhz")

Effective divider n_eff = 175.2
Effective output frequency = 175.5 x 10 MHz = 1752.0 Mhz


To reach the 1.752 GHz, the dividor ratios are 175 and 176 with a duty cycle of 20% for 175.